In [154]:
'''
The script collects all types of pre-processed features 
and outputs a feature set with all 235 features and corresponding values

'''

In [ ]:
import pandas as pd
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics import tsaplots

import netCDF4 as nc
import numpy as np
import xarray as xr
import pandas as pd
from eofs.xarray import Eof

import matplotlib
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
import cartopy as cart
import cartopy.mpl.ticker as cticker
from cartopy.util import add_cyclic_point


from sklearn.preprocessing import StandardScaler
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering

import pickle
import joblib

In [3]:
### label the target ###
### 86 * 40 targets ###

dataset = pd.read_csv(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\result\cluster_data\input_output.csv', index_col="Time")
dataset = dataset.drop(columns=['Unnamed: 0'])

dataset_temp = dataset.drop(index = '1919-12-01 00:00:00',axis = 1)
dataset_new = dataset_temp

In [155]:
### read all variables first ###
# SST TP #
sst_tp1 = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_sst\TP\sst_pc1.nc'
                           )
sst_tp2 = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_sst\TP\sst_pc2.nc'
                           )
sst_tp3= xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_sst\TP\sst_pc3.nc'
                           )
sst_tp4 = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_sst\TP\sst_pc4.nc'
                           )

# SST NA #
sst_na = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_sst\NA\sst_pc_na.nc'
                           )


# SLP #
slp = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_slp\slp_pc_na.nc'
                           )

# Z500 NA #

z500_na = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_z500\z500_pc4_NA.nc'
                           )

# Z500 NP #
z500_np = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_z500\z500_pc4_NP.nc'
                           )

# U200 NA #
u200_na = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_u200\u200_pc_NP.nc'
                           )

# U200 NP #
u200_np = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_u200\u200_pc_NP.nc'
                           )

# IOD #
iod = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\iod\iod.nc'
                           )

# PDO #
pdo = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\PDO\pdo_pc.nc'
                           )

# AMO #
amo = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\amo\amo.nc'
                           )

# SWC #
swc = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\ensemble_data\swc\swc_mean_EU_new.nc')


# SCE EU #
sce_eu = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\ensemble_data\sce\sce_mean_EU.nc')


# SCE CAEA #
sce_caea = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\ensemble_data\sce\sce_mean_CAEA.nc')


# SCE SB #
sce_sb = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\ensemble_data\sce\sce_mean_SB.nc')


# SIC BARANTS #
sic_barants = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\ensemble_data\sic\sic_mean_barants.nc')


# SIC KARA #
sic_kara = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\ensemble_data\sic\sic_mean_kara.nc')



In [156]:
# NAM #
nam = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\NAM\nam_pc.nc'
                           )

# NAO #
nao = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\NAO\nao_pc.nc'
                           )

In [179]:
### standarization and dataframe ###

scaler_sst_na = StandardScaler()
sst_na_nor = scaler_sst_na.fit_transform(sst_na['pcs'].values)
sst_na_df = pd.DataFrame(sst_na_nor, index = pd.Series(sst_na['pcs'].time),dtype=float) # columns=['var']


scaler_slp = StandardScaler()
slp_nor = scaler_slp.fit_transform(slp['pcs'].values)
slp_df = pd.DataFrame(slp_nor, index = pd.Series(slp['pcs'].time),dtype=float) # columns=['var']


scaler_z500_na = StandardScaler()
z500_na_nor = scaler_z500_na.fit_transform(z500_na['pcs'].values)
z500_na_df = pd.DataFrame(z500_na_nor, index = pd.Series(sst_na['pcs'].time),dtype=float) # columns=['var']


scaler_z500_np = StandardScaler()
z500_np_nor = scaler_z500_np.fit_transform(z500_np['pcs'].values)
z500_np_df = pd.DataFrame(z500_np_nor, index = pd.Series(sst_na['pcs'].time),dtype=float) # columns=['var']


scaler_u200_na = StandardScaler()
u200_na_nor = scaler_u200_na.fit_transform(u200_na['pcs'].values)
u200_na_df = pd.DataFrame(u200_na_nor, index = pd.Series(sst_na['pcs'].time),dtype=float) # columns=['var']


scaler_u200_np = StandardScaler()
u200_np_nor = scaler_u200_np.fit_transform(u200_np['pcs'].values)
u200_np_df = pd.DataFrame(u200_np_nor, index = pd.Series(sst_na['pcs'].time),dtype=float) # columns=['var']

scaler_iod = StandardScaler()
iod_nor = scaler_iod.fit_transform(iod['__xarray_dataarray_variable__'].values.reshape(-1,1))
iod_df = pd.DataFrame(iod_nor, 
                      index = pd.Series(iod['__xarray_dataarray_variable__'].time),dtype=float) # columns=['var']


scaler_pdo = StandardScaler()
pdo_nor = scaler_pdo.fit_transform(pdo['pcs'].values)
pdo_df = pd.DataFrame(pdo_nor, 
                      index = pd.Series(pdo['pcs'].time),dtype=float) # columns=['var']

scaler_amo = StandardScaler()
amo_nor = scaler_amo.fit_transform(amo['__xarray_dataarray_variable__'].values.reshape(-1,1))
amo_df = pd.DataFrame(amo_nor, 
                      index = pd.Series(amo['__xarray_dataarray_variable__'].time),dtype=float) # columns=['var']

scaler_swc = StandardScaler()
swc_nor = scaler_swc.fit_transform(swc['__xarray_dataarray_variable__'].values.reshape(-1,1))
swc_df = pd.DataFrame(swc_nor, 
                      index = pd.Series(swc['__xarray_dataarray_variable__'].time),dtype=float) # columns=['var']

scaler_sce_eu = StandardScaler()
sce_eu_nor = scaler_sce_eu.fit_transform(sce_eu['__xarray_dataarray_variable__'].values.reshape(-1,1))
sce_eu_df = pd.DataFrame(sce_eu_nor, 
                      index = pd.Series(sce_eu['__xarray_dataarray_variable__'].time),dtype=float) # columns=['var']

scaler_sce_caea = StandardScaler()
sce_caea_nor = scaler_sce_caea.fit_transform(sce_caea['__xarray_dataarray_variable__'].values.reshape(-1,1))
sce_caea_df = pd.DataFrame(sce_caea_nor, 
                      index = pd.Series(sce_caea['__xarray_dataarray_variable__'].time),dtype=float) # columns=['var']

scaler_sce_sb = StandardScaler()
sce_sb_nor = scaler_sce_sb.fit_transform(sce_sb['__xarray_dataarray_variable__'].values.reshape(-1,1))
sce_sb_df = pd.DataFrame(sce_sb_nor, 
                      index = pd.Series(sce_sb['__xarray_dataarray_variable__'].time),dtype=float) # columns=['var']

scaler_sic_barants = StandardScaler()
sic_barants_nor = scaler_sic_barants.fit_transform(sic_barants['__xarray_dataarray_variable__'].values.reshape(-1,1))
sic_barants_df = pd.DataFrame(sic_barants_nor, 
                      index = pd.Series(sic_barants['__xarray_dataarray_variable__'].time),dtype=float) # columns=['var']

scaler_sic_kara = StandardScaler()
sic_kara_nor = scaler_sic_kara.fit_transform(sic_kara['__xarray_dataarray_variable__'].values.reshape(-1,1))
sic_kara_df = pd.DataFrame(sic_kara_nor, 
                      index = pd.Series(sic_kara['__xarray_dataarray_variable__'].time),dtype=float) # columns=['var']


scaler_nam = StandardScaler()
nam_nor = scaler_nam.fit_transform(nam['pcs'].values)
nam_df = pd.DataFrame(nam_nor, 
                      index = pd.Series(nam['pcs'].time),dtype=float) # columns=['var']

scaler_nao = StandardScaler()
nao_nor = scaler_nao.fit_transform(nao['pcs'].values)
nao_df = pd.DataFrame(nao_nor, 
                      index = pd.Series(nao['pcs'].time),dtype=float) # columns=['var']

In [158]:
# Nino #

nino1_2=pd.read_csv(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\NINO\nino_1_2.csv'
                    ,index_col=0)

nino3=pd.read_csv(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\NINO\nino_3.csv'
                    ,index_col=0)


nino4=pd.read_csv(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\NINO\nino_4.csv'
                    ,index_col=0)

nino3_4=pd.read_csv(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\NINO\nino_3_4.csv'
                    ,index_col=0)


scaler_nino1_2 = StandardScaler()
nino1_2_nor = scaler_nino1_2.fit_transform(nino1_2.values.flatten().reshape(-1,1))
nino1_2_df = pd.DataFrame(nino1_2_nor, 
                      index = pd.Series(sic_kara['__xarray_dataarray_variable__'].time),dtype=float) # columns=['var']

scaler_nino3 = StandardScaler()
nino3_nor = scaler_nino3.fit_transform(nino3.values.flatten().reshape(-1,1))
nino3_df = pd.DataFrame(nino3_nor, 
                      index = pd.Series(sic_kara['__xarray_dataarray_variable__'].time),dtype=float) # columns=['var']

scaler_nino4 = StandardScaler()
nino4_nor = scaler_nino4.fit_transform(nino4.values.flatten().reshape(-1,1))
nino4_df = pd.DataFrame(nino4_nor, 
                      index = pd.Series(sic_kara['__xarray_dataarray_variable__'].time),dtype=float) # columns=['var']


scaler_nino3_4 = StandardScaler()
nino3_4_nor = scaler_nino3_4.fit_transform(nino3_4.values.flatten().reshape(-1,1))
nino3_4_df = pd.DataFrame(nino3_4_nor, 
                      index = pd.Series(sic_kara['__xarray_dataarray_variable__'].time),dtype=float) # columns=['var']

In [159]:
### sst-tp-df ###
sst_tp_xr = xr.concat([sst_tp1['pcs'], sst_tp2['pcs'], sst_tp3['pcs'],sst_tp4['pcs']], dim="time")

scaler_sst_tp = StandardScaler()
sst_tp_nor = scaler_sst_tp.fit_transform(sst_tp_xr.values)
sst_tp_xr_df = pd.DataFrame(sst_tp_nor, index = pd.Series(sst_tp_xr.time),dtype=float) # columns=['var']

In [86]:
### seasonal data ###
# SST TP #
sst_tp_djf = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_sst\TP\sst_pc_djf_tp.nc'
                           )
scaler_sst_tp_djf = StandardScaler()
sst_tp_djf_nor = scaler_sst_tp_djf.fit_transform(sst_tp_djf['pcs'].values)
sst_tp_djf_df = pd.DataFrame(sst_tp_djf_nor, index = pd.Series(sst_tp_djf['pcs'].time),dtype=float) # columns=['var']


sst_tp_mam = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_sst\TP\sst_pc_mam_tp.nc'
                           )
scaler_sst_tp_mam = StandardScaler()
sst_tp_mam_nor = scaler_sst_tp_djf.fit_transform(sst_tp_mam['pcs'].values)
sst_tp_mam_df = pd.DataFrame(sst_tp_mam_nor, index = pd.Series(sst_tp_mam['pcs'].time),dtype=float) # columns=['var']

sst_tp_jja = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_sst\TP\sst_pc_jja_tp.nc'
                           )
scaler_sst_tp_jja = StandardScaler()
sst_tp_jja_nor = scaler_sst_tp_jja.fit_transform(sst_tp_jja['pcs'].values)
sst_tp_jja_df = pd.DataFrame(sst_tp_jja_nor, index = pd.Series(sst_tp_jja['pcs'].time),dtype=float) # columns=['var']

sst_tp_son = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_sst\TP\sst_pc_son_tp.nc'
                           )
scaler_sst_tp_son = StandardScaler()
sst_tp_son_nor = scaler_sst_tp_son.fit_transform(sst_tp_son['pcs'].values)
sst_tp_son_df = pd.DataFrame(sst_tp_son_nor, index = pd.Series(sst_tp_son['pcs'].time),dtype=float) # columns=['var']

# SST NA #
sst_na_djf = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_sst\NA\sst_pc_djf_na.nc'
                           )
scaler_sst_na_djf = StandardScaler()
sst_na_djf_nor = scaler_sst_na_djf.fit_transform(sst_na_djf['pcs'].values)
sst_na_djf_df = pd.DataFrame(sst_na_djf_nor, index = pd.Series(sst_na_djf['pcs'].time),dtype=float) # columns=['var']


sst_na_mam = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_sst\NA\sst_pc_mam_na.nc'
                           )
scaler_sst_na_mam = StandardScaler()
sst_na_mam_nor = scaler_sst_na_djf.fit_transform(sst_na_mam['pcs'].values)
sst_na_mam_df = pd.DataFrame(sst_na_mam_nor, index = pd.Series(sst_na_mam['pcs'].time),dtype=float) # columns=['var']

sst_na_jja = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_sst\NA\sst_pc_jja_na.nc'
                           )
scaler_sst_na_jja = StandardScaler()
sst_na_jja_nor = scaler_sst_na_jja.fit_transform(sst_na_jja['pcs'].values)
sst_na_jja_df = pd.DataFrame(sst_na_jja_nor, index = pd.Series(sst_na_jja['pcs'].time),dtype=float) # columns=['var']

sst_na_son = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_sst\NA\sst_pc_son_na.nc'
                           )
scaler_sst_na_son = StandardScaler()
sst_na_son_nor = scaler_sst_na_son.fit_transform(sst_na_son['pcs'].values)
sst_na_son_df = pd.DataFrame(sst_na_son_nor, index = pd.Series(sst_na_son['pcs'].time),dtype=float) # columns=['var']
# SLP #
slp_djf = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_slp\slp_pc_djf.nc'
                           )
scaler_slp_djf = StandardScaler()
slp_djf_nor = scaler_slp_djf.fit_transform(slp_djf['pcs'].values)
slp_djf_df = pd.DataFrame(slp_djf_nor, index = pd.Series(slp_djf['pcs'].time),dtype=float) # columns=['var']

slp_mam = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_slp\slp_pc_mam.nc'
                           )
scaler_slp_mam = StandardScaler()
slp_mam_nor = scaler_slp_mam.fit_transform(slp_mam['pcs'].values)
slp_mam_df = pd.DataFrame(slp_mam_nor, index = pd.Series(slp_mam['pcs'].time),dtype=float) # columns=['var']

slp_jja = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_slp\slp_pc_jja.nc'
                           )
scaler_slp_jja = StandardScaler()
slp_jja_nor = scaler_slp_djf.fit_transform(slp_jja['pcs'].values)
slp_jja_df = pd.DataFrame(slp_jja_nor, index = pd.Series(slp_jja['pcs'].time),dtype=float) # columns=['var']

slp_son = xr.open_dataset(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\processed_data\eof_slp\slp_pc_son.nc'
                           )
scaler_slp_son = StandardScaler()
slp_son_nor = scaler_slp_djf.fit_transform(slp_son['pcs'].values)
slp_son_df = pd.DataFrame(slp_son_nor, index = pd.Series(slp_son['pcs'].time),dtype=float) 


In [188]:
### COMBINE ALL MONTHLY DATA TO dataframe ###
month_data = pd.concat([sst_tp_xr_df.iloc[:,0:4],sst_na_df.iloc[:,0:4],slp_df.iloc[:,0:4],z500_na_df.iloc[:,0:4],z500_np_df.iloc[:,0:4]
          ,u200_na_df.iloc[:,0:4],u200_np_df.iloc[:,0:4],iod_df, pdo_df.iloc[:,0],amo_df, sic_barants_df
        ,sic_kara_df,sce_eu_df, sce_caea_df, sce_sb_df, nino1_2_df, nino3_4_df,nino3_df, nino4_df]
          ,axis=1)

name = ['sst_tp_pc1','sst_tp_pc2','sst_tp_pc3','sst_tp_pc4','sst_na_pc1'
           ,'sst_na_pc2'
           ,'sst_na_pc3'
           ,'sst_na_pc4'
           ,'slp_pc1','slp_pc2'
           ,'slp_pc3','slp_pc4'
            ,'z500_na_pc1','z500_na_pc2'
           ,'z500_na_pc3','z500_na_pc4','z500_np_pc1','z500_np_pc2'
           ,'z500_np_pc3','z500_np_pc4','u200_na_pc1','u200_na_pc2','u200_na_pc3'
           ,'u200_na_pc4','u200_np_pc1','u200_np_pc2','u200_np_pc3','u200_np_pc4'
            ,'iod','pdo'
            ,'amo','sce_eu','sic_barents','sic_kara','sce_caea','sce_sb'
            ,'nino12','nino34','nino3','nino4']

month_data_new = pd.DataFrame(month_data.values, index = pd.Series(sst_na['pcs'].time),columns=name,dtype=float)

In [198]:
### COMBINE ALL SEASONAL DATA TO dataframe ###
season_data = np.concatenate([sst_tp_djf_df.iloc[:,0:4].values, sst_tp_mam_df.iloc[:,0:4].values
               ,sst_tp_jja_df.iloc[:,0:4].values,sst_tp_son_df.iloc[:,0:4].values
              ,sst_na_djf_df.iloc[:,0:4].values, sst_na_mam_df.iloc[:,0:4].values
               ,sst_na_jja_df.iloc[:,0:4].values,sst_na_son_df.iloc[:,0:4].values
               ,slp_djf_df.iloc[:,0:4].values,slp_mam_df.iloc[:,0:4].values
               ,slp_jja_df.iloc[:,0:4].values,slp_son_df.iloc[:,0:4].values],axis =1)

season_name = ['sst_tp_djf_pc1','sst_tp_djf_pc2','sst_tp_djf_pc3','sst_tp_djf_pc4'
            ,'sst_tp_mam_pc1','sst_tp_mam_pc2','sst_tp_mam_pc3','sst_tp_mam_pc4'
            ,'sst_tp_jja_pc1','sst_tp_jja_pc2','sst_tp_jja_pc3','sst_tp_jja_pc4'
            ,'sst_tp_son_pc1','sst_tp_son_pc2','sst_tp_son_pc3','sst_tp_son_pc4'
            ,'sst_na_djf_pc1','sst_na_djf_pc2','sst_na_djf_pc3','sst_na_djf_pc4'
            ,'sst_na_mam_pc1','sst_na_mam_pc2','sst_na_mam_pc3','sst_na_mam_pc4'
            ,'sst_na_jja_pc1','sst_na_jja_pc2','sst_na_jja_pc3','sst_na_jja_pc4'
            ,'sst_na_son_pc1','sst_na_son_pc2','sst_na_son_pc3','sst_na_son_pc4'
            ,'slp_djf_pc1','slp_djf_pc2','slp_djf_pc3','slp_djf_pc4'
            ,'slp_mam_pc1','slp_mam_pc2','slp_mam_pc3','slp_mam_pc4'
            ,'slp_jja_pc1','slp_jja_pc2','slp_jja_pc3','slp_jja_pc4'
            ,'slp_son_pc1','slp_son_pc2','slp_son_pc3','slp_son_pc4']

season_data_new = pd.DataFrame(season_data, index = pd.Series(sst_tp_djf['pcs'].time),columns=season_name,dtype=float)


In [149]:
features = ['sst_tp_pc1_lag1','sst_tp_pc1_lag2','sst_tp_pc1_lag3','sst_tp_pc1_lag4','sst_tp_pc1_lag5','sst_tp_pc1_lag6'
           ,'sst_tp_pc1_lag7','sst_tp_pc1_lag8','sst_tp_pc1_lag9','sst_tp_pc1_lag10','sst_tp_pc1_lag11','sst_tp_pc1_lag12'
           ,'sst_tp_pc2_lag1','sst_tp_pc2_lag2','sst_tp_pc2_lag3','sst_tp_pc2_lag4','sst_tp_pc2_lag5','sst_tp_pc2_lag6'
           ,'sst_tp_pc2_lag7','sst_tp_pc2_lag8','sst_tp_pc2_lag9','sst_tp_pc2_lag10','sst_tp_pc2_lag11','sst_tp_pc2_lag12'
           ,'sst_tp_pc3_lag1','sst_tp_pc3_lag2','sst_tp_pc3_lag3','sst_tp_pc3_lag4','sst_tp_pc3_lag5','sst_tp_pc3_lag6'
           ,'sst_tp_pc4_lag1','sst_tp_pc4_lag2','sst_tp_pc4_lag3','sst_tp_pc4_lag4','sst_tp_pc4_lag5','sst_tp_pc4_lag6'
           ,'sst_na_pc1_lag1','sst_na_pc1_lag2','sst_na_pc1_lag3','sst_na_pc1_lag4','sst_na_pc1_lag5','sst_na_pc1_lag6'
           ,'sst_na_pc1_lag7','sst_na_pc1_lag8','sst_na_pc1_lag9','sst_na_pc1_lag10','sst_na_pc1_lag11','sst_na_pc1_lag12'
           ,'sst_na_pc2_lag1','sst_na_pc2_lag2','sst_na_pc2_lag3','sst_na_pc2_lag4','sst_na_pc2_lag5','sst_na_pc2_lag6'
           ,'sst_na_pc3_lag1','sst_na_pc3_lag2','sst_na_pc3_lag3','sst_na_pc3_lag4','sst_na_pc3_lag5','sst_na_pc3_lag6'
           ,'sst_na_pc4_lag1','sst_na_pc4_lag2','sst_na_pc4_lag3','sst_na_pc4_lag4','sst_na_pc4_lag5','sst_na_pc4_lag6'
           ,'slp_pc1_lag1','slp_pc2_lag1','slp_pc2_lag2','slp_pc2_lag3','slp_pc2_lag4','slp_pc2_lag5','slp_pc2_lag6'
           ,'slp_pc3_lag1','slp_pc4_lag1'
            ,'z500_na_pc1_lag1','z500_na_pc1_lag2','z500_na_pc1_lag3','z500_na_pc1_lag4','z500_na_pc1_lag5'
           ,'z500_na_pc1_lag6','z500_na_pc1_lag7','z500_na_pc1_lag8','z500_na_pc1_lag9','z500_na_pc1_lag10'
           ,'z500_na_pc1_lag11','z500_na_pc1_lag12','z500_na_pc2_lag1'#,'z500_na_pc2_lag2','z500_na_pc2_lag3'
           ,'z500_na_pc3_lag1','z500_na_pc4_lag1','z500_np_pc1_lag1','z500_np_pc1_lag2','z500_np_pc2_lag1'
           ,'z500_np_pc3_lag1','z500_np_pc4_lag1','u200_na_pc1_lag1','u200_na_pc1_lag2','u200_na_pc1_lag3'
           ,'u200_na_pc1_lag4','u200_na_pc1_lag5','u200_na_pc1_lag6','u200_na_pc2_lag1','u200_na_pc3_lag1'
           ,'u200_na_pc4_lag1','u200_np_pc1_lag1','u200_np_pc1_lag2','u200_np_pc2_lag1','u200_np_pc3_lag1'
           ,'u200_np_pc4_lag1'
            ,'swc_lag1','sce_eu_lag1','sce_eu_lag2','sce_caea_lag1','sce_caea_lag2','sce_sb_lag1'
            ,'sic_barents_lag1','sic_barents_lag2','sic_barents_lag3','sic_barents_lag4','sic_barents_lag5'
            ,'sic_barents_lag6','sic_barents_lag7','sic_barents_lag8','sic_barents_lag9','sic_barents_lag10'
            ,'sic_barents_lag11','sic_barents_lag12','sic_kara_lag1','sic_kara_lag2','sic_kara_lag3'
            ,'iod_lag1','iod_lag2','iod_lag3','iod_lag4','iod_lag5','iod_lag6','pdo_lag1','pdo_lag2','pdo_lag3','pdo_lag4'
           ,'pdo_lag5','pdo_lag6','pdo_lag7','pdo_lag8','pdo_lag9','pdo_lag10','pdo_lag11','pdo_lag12'
            ,'amo_lag1','amo_lag2','amo_lag3','amo_lag4','amo_lag5','amo_lag6','amo_lag7','amo_lag8','amo_lag9','amo_lag10'
           ,'amo_lag11','amo_lag12','nino12_lag1','nino12_lag2','nino12_lag3','nino12_lag4','nino12_lag5','nino12_lag6'
           ,'nino34_lag1','nino34_lag2','nino34_lag3','nino34_lag4','nino34_lag5','nino34_lag6','nino3_lag1','nino3_lag2'
           ,'nino3_lag3','nino3_lag4','nino3_lag5','nino3_lag6','nino4_lag1','nino4_lag2','nino4_lag3','nino4_lag4'
           ,'nino4_lag5','nino4_lag6'
            ,'sst_tp_djf_pc1','sst_tp_djf_pc2','sst_tp_djf_pc3','sst_tp_djf_pc4'
            ,'sst_tp_mam_pc1','sst_tp_mam_pc2','sst_tp_mam_pc3','sst_tp_mam_pc4'
            ,'sst_tp_jja_pc1','sst_tp_jja_pc2','sst_tp_jja_pc3','sst_tp_jja_pc4'
            ,'sst_tp_son_pc1','sst_tp_son_pc2','sst_tp_son_pc3','sst_tp_son_pc4'
            ,'sst_na_djf_pc1','sst_na_djf_pc2','sst_na_djf_pc3','sst_na_djf_pc4'
            ,'sst_na_mam_pc1','sst_na_mam_pc2','sst_na_mam_pc3','sst_na_mam_pc4'
            ,'sst_na_jja_pc1','sst_na_jja_pc2','sst_na_jja_pc3','sst_na_jja_pc4'
            ,'sst_na_son_pc1','sst_na_son_pc2','sst_na_son_pc3','sst_na_son_pc4'
            ,'slp_djf_pc1','slp_djf_pc2','slp_djf_pc3','slp_djf_pc4'
            ,'slp_mam_pc1','slp_mam_pc2','slp_mam_pc3','slp_mam_pc4'
            ,'slp_jja_pc1','slp_jja_pc2','slp_jja_pc3','slp_jja_pc4'
            ,'slp_son_pc1','slp_son_pc2','slp_son_pc3','slp_son_pc4'
            #,'z500_np_month_pc1','z500_np_month_pc2','z500_np_month_pc3','z500_np_month_pc4'
            #,'z500_na_month_pc1','z500_na_month_pc2','z500_na_month_pc3','z500_na_month_pc4'
            #,'u200_na_month_pc1','u200_na_month_pc2','u200_na_month_pc3','u200_na_month_pc4'
            #,'u200_np_month_pc1','u200_np_month_pc2','u200_np_month_pc3','u200_np_month_pc4'
            ,'nam','nao','cluster_lag1']

### empty dataframe for storing the features ###
feature_set = pd.DataFrame(index = pd.Series(dataset_new.index),columns=features, dtype=float) # 

In [150]:
### feature set ###
for i in range(3440): # for each year -- concatenate the features
    if i % 86 == 0:
        # sst-tp-pc1-lag1-12
        feature_set.iloc[i,0] = sst_tp_xr_df.iloc[i*12,0]
        feature_set.iloc[i,1:12] = sst_tp_xr_df.iloc[(i*12):(11+i*12),0]
        # sst-tp-pc2-lag1-12
        feature_set.iloc[i,12] = sst_tp_xr_df.iloc[i*12,1]
        feature_set.iloc[i,13:24] = sst_tp_xr_df.iloc[(i*12):(11+i*12),1]
        # sst-tp-pc3-lag1-6
        feature_set.iloc[i,24:30] = sst_tp_xr_df.iloc[(i*12+5):(11+i*12),2]
        # sst-tp-pc4-lag1-6
        feature_set.iloc[i,30:36] = sst_tp_xr_df.iloc[(i*12+5):(11+i*12),3]
        # sst-na-pc1-lag1-12
        feature_set.iloc[i,36] = sst_na_df.iloc[i*12,0]
        feature_set.iloc[i,37:48] = sst_na_df.iloc[(i*12):(11+i*12),0]
        # sst-na-pc2-lag1-6
        feature_set.iloc[i,48:54] = sst_na_df.iloc[(i*12+5):(11+i*12),1]
        # sst-na-pc3-lag1-6
        feature_set.iloc[i,54:60] = sst_na_df.iloc[(i*12+5):(11+i*12),2]
        # sst-na-pc4-lag1-6
        feature_set.iloc[i,60:66] = sst_na_df.iloc[(i*12+5):(11+i*12),3]
        # slp-pc1-lag1
        feature_set.iloc[i,66] = slp_df.iloc[(10+i*12),0]
        # slp-pc2-lag1-6
        feature_set.iloc[i,67:73] = slp_df.iloc[(i*12+5):(11+i*12),1]
        # slp-pc3-lag1
        feature_set.iloc[i,73] = slp_df.iloc[(10+i*12),2]
        # slp-pc4-lag1
        feature_set.iloc[i,74] = slp_df.iloc[(10+i*12),3]
        # z500-na-pc1-lag12
        feature_set.iloc[i,75] = z500_na_df.iloc[i*12,0]
        feature_set.iloc[i,76:87] = z500_na_df.iloc[(i*12):(11+i*12),0]
        # z500-na-pc2-lag1
        feature_set.iloc[i,87] = z500_na_df.iloc[(10+i*12),1]
        # z500-na-pc3-lag1
        feature_set.iloc[i,88] = z500_na_df.iloc[(10+i*12),2]
        # z500-na-pc4-lag1
        feature_set.iloc[i,89] = z500_na_df.iloc[(10+i*12),3]
        # z500-np-pc1-lag2
        feature_set.iloc[i,90:92] = z500_np_df.iloc[(9+i*12):(11+i*12),0]
        # z500-np-pc2-lag1
        feature_set.iloc[i,92] = z500_np_df.iloc[(10+i*12),1]
        # z500-np-pc3-lag1
        feature_set.iloc[i,93] = z500_np_df.iloc[(10+i*12),2]
        # z500-np-pc4-lag1
        feature_set.iloc[i,94] = z500_np_df.iloc[(10+i*12),3]
        # u200-na-pc1-lag6
        feature_set.iloc[i,95:101] = u200_na_df.iloc[(i*12+5):(11+i*12),0]
        # u200-na-pc2-lag1
        feature_set.iloc[i,101] = u200_na_df.iloc[(10+i*12),1]
        # u200-na-pc3-lag1
        feature_set.iloc[i,102] = u200_na_df.iloc[(10+i*12),2]
        # u200-na-pc4-lag1
        feature_set.iloc[i,103] = u200_na_df.iloc[(10+i*12),3]
        # u200-np-pc1-lag2
        feature_set.iloc[i,104:106] = u200_np_df.iloc[(9+i*12):(11+i*12),0]
        # u200-np-pc2-lag1
        feature_set.iloc[i,106] = u200_np_df.iloc[(10+i*12),1]
        # u200-np-pc3-lag1
        feature_set.iloc[i,107] = u200_np_df.iloc[(10+i*12),2]
        # u200-np-pc4-lag1
        feature_set.iloc[i,108] = u200_np_df.iloc[(10+i*12),3]
        #swc-lag1
        feature_set.iloc[i,109] = swc_df.iloc[(10+i*12),0]
        #sce-eu-lag2
        feature_set.iloc[i,110:112] = sce_eu_df.iloc[(9+i*12):(11+i*12),0]
        #sce-caea-lag2
        feature_set.iloc[i,112:114] = sce_caea_df.iloc[(9+i*12):(11+i*12),0]
        #sce-sb-lag1
        feature_set.iloc[i,114] = sce_sb_df.iloc[(10+i*12),0]
        # sic-ba-lag12
        feature_set.iloc[i,115] = sic_barants_df.iloc[i*12,0]
        feature_set.iloc[i,116:127] = sic_barants_df.iloc[(i*12):(11+i*12),0]
        #sic-kara-lag3
        feature_set.iloc[i,127:130] = sic_kara_df.iloc[(8+i*12):(11+i*12),0]
        #iod-lag6
        feature_set.iloc[i,130:136] = iod_df.iloc[(5+i*12):(11+i*12),0]
        #pdo-lag12
        feature_set.iloc[i,136] = pdo_df.iloc[i*12,0]
        feature_set.iloc[i,137:148] = pdo_df.iloc[(i*12):(11+i*12),0]
        #amo-lag12
        feature_set.iloc[i,148] = amo_df.iloc[i*12,0]
        feature_set.iloc[i,149:160] = amo_df.iloc[(i*12):(11+i*12),0]
        #nino1-2-lag6
        feature_set.iloc[i,160:166] = nino1_2_df.iloc[(5+i*12):(11+i*12),0]
        #nino3-4-lag6
        feature_set.iloc[i,166:172] = nino3_4_df.iloc[(5+i*12):(11+i*12),0]
        #nino3-lag6
        feature_set.iloc[i,172:178] = nino3_df.iloc[(5+i*12):(11+i*12),0]
        #nino4-lag6
        feature_set.iloc[i,178:184] = nino4_df.iloc[(5+i*12):(11+i*12),0]
        #sst-tp-djf-pc1-4
        feature_set.iloc[i,184:188] = sst_tp_djf_df.iloc[i,0:4]
        #sst-tp-mam-pc1-4
        feature_set.iloc[i,188:192] = sst_tp_mam_df.iloc[i,0:4]
        #sst-tp-jja-pc1-4
        feature_set.iloc[i,192:196] = sst_tp_jja_df.iloc[i,0:4]
        #sst-tp-son-pc1-4
        feature_set.iloc[i,196:200] = sst_tp_son_df.iloc[i,0:4]
        #sst-na-djf-pc1-4
        feature_set.iloc[i,200:204] = sst_na_djf_df.iloc[i,0:4]
        #sst-na-mam-pc1-4
        feature_set.iloc[i,204:208] = sst_na_mam_df.iloc[i,0:4]
        #sst-na-jja-pc1-4
        feature_set.iloc[i,208:212] = sst_na_jja_df.iloc[i,0:4]
        #sst-na-son-pc1-4
        feature_set.iloc[i,212:216] = sst_na_son_df.iloc[i,0:4]
        #slp-djf-pc1-4
        feature_set.iloc[i,216:220] = slp_djf_df.iloc[i,0:4]
        #slp-mam-pc1-4
        feature_set.iloc[i,220:224] = slp_mam_df.iloc[i,0:4]
        #slp-jja-pc1-4
        feature_set.iloc[i,224:228] = slp_jja_df.iloc[i,0:4]
        #slp-son-pc1-4
        feature_set.iloc[i,228:232] = slp_son_df.iloc[i,0:4]
        #nam
        feature_set.iloc[i,232] = nam_df.iloc[i,0]
        #nao
        feature_set.iloc[i,233] = nao_df.iloc[i,0]
        #lag
        feature_set.iloc[i,234] = dataset.iloc[int((i/86)*87)]
        
    else:
        # sst-tp-pc1-lag1-12
        feature_set.iloc[i,:12] = sst_tp_xr_df.iloc[((i*12)-1):(11+i*12),0]
        # sst-tp-pc2-lag1-12
        feature_set.iloc[i,12:24] = sst_tp_xr_df.iloc[((i*12)-1):(11+i*12),1]
        # sst-tp-pc3-lag1-6
        feature_set.iloc[i,24:30] = sst_tp_xr_df.iloc[(i*12+5):(11+i*12),2]
        # sst-tp-pc4-lag1-6
        feature_set.iloc[i,30:36] = sst_tp_xr_df.iloc[(i*12+5):(11+i*12),3]
        # sst-na-pc1-lag1-12
        feature_set.iloc[i,36:48] = sst_na_df.iloc[((i*12)-1):(11+i*12),0]
        # sst-na-pc2-lag1-6
        feature_set.iloc[i,48:54] = sst_na_df.iloc[(i*12+5):(11+i*12),1]
        # sst-na-pc3-lag1-6
        feature_set.iloc[i,54:60] = sst_na_df.iloc[(i*12+5):(11+i*12),2]
        # sst-na-pc4-lag1-6
        feature_set.iloc[i,60:66] = sst_na_df.iloc[(i*12+5):(11+i*12),3]
        # slp-pc1-lag1
        feature_set.iloc[i,66] = slp_df.iloc[(10+i*12),0]
        # slp-pc2-lag1-6
        feature_set.iloc[i,67:73] = slp_df.iloc[(i*12+5):(11+i*12),1]
        # slp-pc3-lag1
        feature_set.iloc[i,73] = slp_df.iloc[(10+i*12),2]
        # slp-pc4-lag1
        feature_set.iloc[i,74] = slp_df.iloc[(10+i*12),3]
        # z500-na-pc1-lag12
        feature_set.iloc[i,75:87] = z500_na_df.iloc[((i*12)-1):(11+i*12),0]
        # z500-na-pc2-lag1
        feature_set.iloc[i,87] = z500_na_df.iloc[(10+i*12),1]
        # z500-na-pc3-lag1
        feature_set.iloc[i,88] = z500_na_df.iloc[(10+i*12),2]
        # z500-na-pc4-lag1
        feature_set.iloc[i,89] = z500_na_df.iloc[(10+i*12),3]
        # z500-np-pc1-lag2
        feature_set.iloc[i,90:92] = z500_np_df.iloc[(9+i*12):(11+i*12),0]
        # z500-np-pc2-lag1
        feature_set.iloc[i,92] = z500_np_df.iloc[(10+i*12),1]
        # z500-np-pc3-lag1
        feature_set.iloc[i,93] = z500_np_df.iloc[(10+i*12),2]
        # z500-np-pc4-lag1
        feature_set.iloc[i,94] = z500_np_df.iloc[(10+i*12),3]
        # u200-na-pc1-lag6
        feature_set.iloc[i,95:101] = u200_na_df.iloc[(i*12+5):(11+i*12),0]
        # u200-na-pc2-lag1
        feature_set.iloc[i,101] = u200_na_df.iloc[(10+i*12),1]
        # u200-na-pc3-lag1
        feature_set.iloc[i,102] = u200_na_df.iloc[(10+i*12),2]
        # u200-na-pc4-lag1
        feature_set.iloc[i,103] = u200_na_df.iloc[(10+i*12),3]
        # u200-np-pc1-lag2
        feature_set.iloc[i,104:106] = u200_np_df.iloc[(9+i*12):(11+i*12),0]
        # u200-np-pc2-lag1
        feature_set.iloc[i,106] = u200_np_df.iloc[(10+i*12),1]
        # u200-np-pc3-lag1
        feature_set.iloc[i,107] = u200_np_df.iloc[(10+i*12),2]
        # u200-np-pc4-lag1
        feature_set.iloc[i,108] = u200_np_df.iloc[(10+i*12),3]
        #swc-lag1
        feature_set.iloc[i,109] = swc_df.iloc[(10+i*12),0]
        #sce-eu-lag2
        feature_set.iloc[i,110:112] = sce_eu_df.iloc[(9+i*12):(11+i*12),0]
        #sce-caea-lag2
        feature_set.iloc[i,112:114] = sce_caea_df.iloc[(9+i*12):(11+i*12),0]
        #sce-sb-lag1
        feature_set.iloc[i,114] = sce_sb_df.iloc[(10+i*12),0]
        # sic-ba-lag12
        feature_set.iloc[i,115:127] = sic_barants_df.iloc[((i*12)-1):(11+i*12),0]
        #sic-kara-lag3
        feature_set.iloc[i,127:130] = sic_kara_df.iloc[(8+i*12):(11+i*12),0]
        #iod-lag6
        feature_set.iloc[i,130:136] = iod_df.iloc[(5+i*12):(11+i*12),0]
        #pdo-lag12
        feature_set.iloc[i,136:148] = pdo_df.iloc[((i*12)-1):(11+i*12),0]
        #amo-lag12
        feature_set.iloc[i,148:160] = amo_df.iloc[((i*12)-1):(11+i*12),0]
        #nino1-2-lag6
        feature_set.iloc[i,160:166] = nino1_2_df.iloc[(5+i*12):(11+i*12),0]
        #nino3-4-lag6
        feature_set.iloc[i,166:172] = nino3_4_df.iloc[(5+i*12):(11+i*12),0]
        #nino3-lag6
        feature_set.iloc[i,172:178] = nino3_df.iloc[(5+i*12):(11+i*12),0]
        #nino4-lag6
        feature_set.iloc[i,178:184] = nino4_df.iloc[(5+i*12):(11+i*12),0]
        #sst-tp-djf-pc1-4
        feature_set.iloc[i,184:188] = sst_tp_djf_df.iloc[i,0:4]
        #sst-tp-mam-pc1-4
        feature_set.iloc[i,188:192] = sst_tp_mam_df.iloc[i,0:4]
        #sst-tp-jja-pc1-4
        feature_set.iloc[i,192:196] = sst_tp_jja_df.iloc[i,0:4]
        #sst-tp-son-pc1-4
        feature_set.iloc[i,196:200] = sst_tp_son_df.iloc[i,0:4]
        #sst-na-djf-pc1-4
        feature_set.iloc[i,200:204] = sst_na_djf_df.iloc[i,0:4]
        #sst-na-mam-pc1-4
        feature_set.iloc[i,204:208] = sst_na_mam_df.iloc[i,0:4]
        #sst-na-jja-pc1-4
        feature_set.iloc[i,208:212] = sst_na_jja_df.iloc[i,0:4]
        #sst-na-son-pc1-4
        feature_set.iloc[i,212:216] = sst_na_son_df.iloc[i,0:4]
        #slp-djf-pc1-4
        feature_set.iloc[i,216:220] = slp_djf_df.iloc[i,0:4]
        #slp-mam-pc1-4
        feature_set.iloc[i,220:224] = slp_mam_df.iloc[i,0:4]
        #slp-jja-pc1-4
        feature_set.iloc[i,224:228] = slp_jja_df.iloc[i,0:4]
        #slp-son-pc1-4
        feature_set.iloc[i,228:232] = slp_son_df.iloc[i,0:4]
        #nam
        feature_set.iloc[i,232] = nam_df.iloc[i,0]
        #nao
        feature_set.iloc[i,233] = nao_df.iloc[i,0]
        #lag
        feature_set.iloc[i,234] = dataset_new.iloc[i-1]


In [152]:
feature_set.to_csv(r'C:\Users\yzh086\OneDrive - University of Bergen\PhD_lit\PhD_lit\task1\result\featureset\features.csv')

In [151]:
feature_set

,sst_tp_pc1_lag1,sst_tp_pc1_lag2,sst_tp_pc1_lag3,sst_tp_pc1_lag4,sst_tp_pc1_lag5,sst_tp_pc1_lag6,sst_tp_pc1_lag7,sst_tp_pc1_lag8,sst_tp_pc1_lag9,sst_tp_pc1_lag10,...,slp_jja_pc2,slp_jja_pc3,slp_jja_pc4,slp_son_pc1,slp_son_pc2,slp_son_pc3,slp_son_pc4,nam,nao,cluster_lag1
Time,,,,,,,,,,,,,,,,,,,,,
1920-12-01 00:00:00,-0.249835,-0.249835,-0.599319,-0.583333,-0.502699,-0.427186,-0.493266,-0.523277,-0.566316,-0.601097,...,-0.278582,1.197571,-0.528416,-0.306985,-0.145836,0.143828,-1.168099,-0.285538,-0.195750,1.0
1921-12-01 00:00:00,-0.224042,-0.215293,-0.357897,-0.164753,-0.016223,0.157646,0.229268,0.339704,0.563819,0.599799,...,0.247980,0.997474,-1.260323,0.524051,0.000468,1.545339,0.682309,0.340124,0.684194,2.0
1922-12-01 00:00:00,0.783974,0.689870,0.714670,0.571732,0.543719,0.507873,0.506387,0.676036,0.804696,0.818897,...,0.035416,0.053099,0.937731,-0.387406,-0.673002,-0.162652,0.943910,0.578964,0.542227,3.0
1923-12-01 00:00:00,0.534357,0.468561,0.662351,0.490527,0.649059,0.813085,0.861565,0.943716,0.918186,0.976903,...,0.681344,-1.839656,0.066171,1.481013,0.070344,-1.248665,1.095878,-0.253233,-0.756776,3.0
1924-12-01 00:00:00,0.734792,0.667698,0.611252,0.542498,0.719592,0.786792,0.863438,0.666473,0.394720,0.119995,...,1.781025,-0.216920,-0.717354,-0.411287,0.057211,0.340638,-0.614778,0.422168,0.241369,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2001-12-01 00:00:00,-0.597775,-0.563335,-0.372085,-0.055800,0.087937,0.173475,0.258534,0.457171,0.476297,0.486263,...,0.181791,0.601539,0.165865,0.408552,-0.466819,-0.732432,-0.887312,-1.410363,-1.299042,1.0
2002-12-01 00:00:00,0.365607,0.243128,-0.053032,-0.098538,-0.175425,-0.196383,-0.122386,0.026856,-0.062594,-0.252592,...,-1.260103,-0.516456,0.456206,-1.239655,-0.267180,0.840673,0.092516,0.274789,1.241297,2.0
2003-12-01 00:00:00,-0.134073,-0.265017,-0.422659,-0.046409,-0.031520,-0.041226,0.097295,0.263617,0.138450,-0.216695,...,0.475503,0.539229,0.088524,0.030264,-0.136791,0.876331,-0.092240,0.862656,0.396375,2.0
